### Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyzbar.pyzbar import decode
from skimage.filters import threshold_local
import os

from local_library.image_operations import *
from sklearn.mixture import GaussianMixture
from sklearn.cluster import DBSCAN
from local_library.ellipsoid import *



In [ ]:
# camera parameter
# logitech c922 pro
# exposure : -4
# high : 65 cm

### Config

In [ ]:
# Configuration for folder paths and file names
LETTUCE_VARIETY_FOLDERS = ['greencos', 'redcos', 'butterhead', 'redbutterhead']
DATASET_FOLDER_PATH = 'D:/#Work/4-1 WiL/Workspace/Dataset/planting_tray/'


# Image and tray dimensions
IMG_HEIGHT_PIXELS, IMG_WIDTH_PIXELS = 420, 600  # Image dimensions in pixels
TRAY_DIMENSIONS_MM = np.array([350, 500])     # Tray shape in millimeters
QR_SIZE_MM = 30                         # QR code size in millimeters
QR_SHIFT_REAL_DISTANCE = 35           # QR code shift in millimeters
# Reference positions for seedlings
SEEDLING_REF_ROW_POS_MM = np.array([90, 267])           # Row positions in millimeters
SEEDLING_REF_COL_POS_MM = np.array([90, 240, 390])      # Column positions in millimeters
NUM_SEEDLING_REFERENCES = len(SEEDLING_REF_ROW_POS_MM) * len(SEEDLING_REF_COL_POS_MM)


# Probability assigned to each position in the tray indicating the prior belief that a seedling could exist there
SEEDLING_POSITION_PRIOR = 0.0000005
# Probability threshold for DBSCAN clustering
DBSCAN_PROBABILITY_THRESHOLD = 0.1
# DBSCAN configuration
IMG_SCALING_RATIO = np.mean([IMG_HEIGHT_PIXELS / 420, IMG_WIDTH_PIXELS / 600])
DBSCAN_MAX_DIST_MM = int(20 * IMG_SCALING_RATIO)       # Maximum distance between points in the same cluster [mm]
DBSCAN_MIN_POINTS = int(95 * (IMG_SCALING_RATIO**2))     # Minimum number of points to form a dense cluster [mm]


# Seedling positioning constraints
MAX_SEEDLING_OFFSET_DISTANCE = 50    # Maximum allowable distance from the reference position for planting in millimeters
MIN_SEEDLING_SIZE = 225               # Minimum required area for a seedling in millimeters


### Main

In [ ]:
# LETTUCE_VARIETY_FOLDERS = ['redbutterhead']
for type_of_lettuce in LETTUCE_VARIETY_FOLDERS:
    
    # sum_have, sum_miss, sum_far, sum_small = 0, 0, 0, 0
    
    # Set the number of Gaussian models based on lettuce type    
    if type_of_lettuce == 'greencos':
        NUM_GAUSSIAN_MODELS = 4
    elif type_of_lettuce == 'redcos':
        NUM_GAUSSIAN_MODELS = 7
    else:
        NUM_GAUSSIAN_MODELS = 5
    
    # Get a sorted list of image names in the folder
    image_filename_list = os.listdir(f'{DATASET_FOLDER_PATH}{type_of_lettuce}/')
    image_filename_list.sort() 
    num_images = len(image_filename_list)

    
    plt.figure(figsize=(42,7*num_images))
    for image_index in range(num_images):
        
        # Read the image
        rgb_image = plt.imread( f'{DATASET_FOLDER_PATH}{type_of_lettuce}/{image_filename_list[image_index]}')

        # Decode QR code from the image
        decoded_qr_results = decodeQR(rgb_image)

        # Initialize positions for tray corners for perspective transformation and define QR order
        qr_centroid_positions = np.zeros((4,2), dtype=int)
        qr_order = [3, 1, 2, 4]
        
        # Store distances between QR code corners
        qr_corner_distances_pixels = []
        for qr_index in range(len(decoded_qr_results)):
            
            # Decode QR data and find its centroid position
            data = np.array(decoded_qr_results[qr_index].data.decode("utf-8").split('.'), dtype=int)
            centroid = np.array(decoded_qr_results[qr_index].polygon, dtype=int).mean(axis=0).astype(int)
            qr_centroid_positions[qr_order.index(data[1])] = centroid
            
            # Calculate distances for each corner and store them
            corner = np.array(decoded_qr_results[qr_index].polygon, dtype=int)
            for corner_index in range (corner.shape[0]):
                qr_corner_distances_pixels.append(calculateEuclidean(corner[corner_index], corner[(corner_index+1)%corner.shape[0]]))
       
        # Calculate millimeters per pixel for full image based on QR size
        mm_per_pixel_full_image = QR_SIZE_MM / np.mean(qr_corner_distances_pixels)

        # Calculate pixel distances between tray corners and convert to real distance
        tray_corner_distances_pixels = []
        for corner_index in range (len(decoded_qr_results)):
            tray_corner_distances_pixels.append(calculateEuclidean(qr_centroid_positions[corner_index], qr_centroid_positions[(corner_index+1)%qr_centroid_positions.shape[0]]))
        tray_corner_distances_mm = np.array(tray_corner_distances_pixels) * mm_per_pixel_full_image
        tray_corner_distances_mm = ((tray_corner_distances_mm[:2] + tray_corner_distances_mm[2:]) / 2)
        tray_corner_distances_mm.sort()
        
        # Calculate mm per pixel specifically for the region of interest (ROI)
        mm_per_pixel_roi = (tray_corner_distances_mm) / np.array([IMG_HEIGHT_PIXELS, IMG_WIDTH_PIXELS])

        # Calculate shifts for each QR code based on actual shift distances and convert to pixel units
        qr_shift_pixels = (QR_SHIFT_REAL_DISTANCE / mm_per_pixel_roi).round().astype(int)
        
        # Apply perspective transformation to warp the original image to the region of interest (ROI) defined by the QR code positions.
        rgb_image_roi = warpPerspectiveTransformation(rgb_image, qr_centroid_positions, IMG_HEIGHT_PIXELS, IMG_WIDTH_PIXELS)

        # Convert the ROI image from RGB to YCbCr color space
        ycbcr_image_roi = convertRGBtoYCbCr(rgb_image_roi)

        # Calculate distances from tray edges for reference
        tray_edge_distances_mm = ((TRAY_DIMENSIONS_MM - tray_corner_distances_mm) / 2)

        # Calculate pixel positions for rows and columns of seedling reference positions in the ROI
        seedling_ref_row_positions_pixels = ((SEEDLING_REF_ROW_POS_MM - tray_edge_distances_mm[0]) / mm_per_pixel_roi[0]).round().astype(int)
        seedling_ref_col_positions_pixels = ((SEEDLING_REF_COL_POS_MM - tray_edge_distances_mm[1]) / mm_per_pixel_roi[1]).round().astype(int)
        seedling_ref_positions_pixels = np.stack(np.meshgrid(seedling_ref_row_positions_pixels, seedling_ref_col_positions_pixels), axis=-1).transpose((1,0,2))

        # Set prior probability in spatial domain and zero in corners due to QR shift
        prior_position_tray = np.ones((IMG_HEIGHT_PIXELS, IMG_WIDTH_PIXELS)) * SEEDLING_POSITION_PRIOR
        prior_position_tray[ : qr_shift_pixels[0]  , : qr_shift_pixels[1]  ] = 0
        prior_position_tray[ : qr_shift_pixels[0]  , - qr_shift_pixels[1] :] = 0
        prior_position_tray[ - qr_shift_pixels[0] :, : qr_shift_pixels[1]  ] = 0
        prior_position_tray[ - qr_shift_pixels[0] :, - qr_shift_pixels[1] :] = 0        
        prior_position_tray[: qr_shift_pixels[0] // 3  ] = 0
        prior_position_tray[- qr_shift_pixels[0] // 3 :] = 0
        prior_position_tray[: , : qr_shift_pixels[1] // 3  ] = 0
        prior_position_tray[: , - qr_shift_pixels[1] // 3 :] = 0

        # Load color models from seedling tray for seedlings and background
        seedling_color_mean = np.loadtxt(f'D:/#Work/4-1 WiL/Workspace/Dataset/planting_tray/seedling_tray/phone/{type_of_lettuce}_mean.npy')
        seedling_color_covariance = np.loadtxt(f'D:/#Work/4-1 WiL/Workspace/Dataset/planting_tray/seedling_tray/phone/{type_of_lettuce}_covariance.npy')
        background_color_means = np.load(f'D:/#Work/4-1 WiL/Workspace/Dataset/planting_tray/seedling_tray/phone/background_{NUM_GAUSSIAN_MODELS-1}_means.npy')
        background_color_covariances = np.load(f'D:/#Work/4-1 WiL/Workspace/Dataset/planting_tray/seedling_tray/phone/background_{NUM_GAUSSIAN_MODELS-1}_covariances.npy')

        # Combine seedling and background color models for the tray
        seedling_tray_color_model_means = np.vstack([seedling_color_mean, background_color_means])
        seedling_tray_color_model_covariances = np.vstack([np.expand_dims(seedling_color_covariance, axis=0), background_color_covariances])

        # Fit a Gaussian Mixture Model (GMM) to the color space of the ROI image
        gmm = GaussianMixture(n_components= NUM_GAUSSIAN_MODELS )
        gmm.fit(ycbcr_image_roi.reshape(-1,3))
        gmm_means = gmm.means_ 
        gmm_covariances = gmm.covariances_
        
        # Align GMM models with existing color models, ensuring the seedling model is first
        pair_gmm_indices = pairGaussianModels(seedling_tray_color_model_means, seedling_tray_color_model_covariances, gmm_means, gmm_covariances)
        gmm_means = gmm_means[pair_gmm_indices[1]]
        gmm_covariances = gmm_covariances[pair_gmm_indices[1]]

        # Compute likelihoods for seedling and background using the GMMs and color models
        pdf_seedling_color_model = calculateMultiGaussianDist(ycbcr_image_roi, seedling_tray_color_model_means[0], seedling_tray_color_model_covariances[0])
        pdf_seedling_gmm = calculateMultiGaussianDist(ycbcr_image_roi, gmm_means[0], gmm_covariances[0])
        foreground_likelihood = pdf_seedling_color_model * pdf_seedling_gmm

        background_likelihood = np.zeros((IMG_HEIGHT_PIXELS, IMG_WIDTH_PIXELS))
        for index_background_clusters in range (1, NUM_GAUSSIAN_MODELS):
            pdf_background_color_model = calculateMultiGaussianDist(ycbcr_image_roi, seedling_tray_color_model_means[index_background_clusters], seedling_tray_color_model_covariances[index_background_clusters])
            pdf_background_gmm = calculateMultiGaussianDist(ycbcr_image_roi, gmm_means[index_background_clusters], gmm_covariances[index_background_clusters])
            background_likelihood += pdf_background_color_model * pdf_background_gmm 

        # Calculate the seedling probability for each cell using Bayes' theorem
        seedling_probability_all_cells = calculateBayes(foreground_likelihood * prior_position_tray, background_likelihood * (1 - prior_position_tray) / (NUM_GAUSSIAN_MODELS-1))

        # Identify pixel positions with a high likelihood of containing seedlings
        seedling_pixel_position = np.stack(np.where(seedling_probability_all_cells > DBSCAN_PROBABILITY_THRESHOLD), axis = -1)

        # Check if there are detected seedlings; if so, apply DBSCAN for clustering
        if len(seedling_pixel_position) != 0:
            # Apply DBSCAN clustering on seedling pixel positions to group seedlings
            dbscan = DBSCAN(eps = DBSCAN_MAX_DIST_MM, min_samples = DBSCAN_MIN_POINTS)
            labels = dbscan.fit_predict(seedling_pixel_position)
            # Calculate the number of distinct seedlings
            number_of_seedling = labels.max() + 1
            if number_of_seedling != 0:
                # Initialize arrays to store means and covariances for each detected seedling
                dbscan_position_means = np.zeros((number_of_seedling, 2))
                dbscan_position_covariances = np.zeros((number_of_seedling, 2, 2))

                # Loop through each detected seedling cluster
                for label_index in range(number_of_seedling):
                    # Get positions of the pixels for the current seedling cluster
                    label_position = seedling_pixel_position[np.where(labels == label_index)]
                    # Calculate mean and covariance for current seedling cluster based on probabilities
                    dbscan_position_means[label_index], dbscan_position_covariances[label_index] = calculateMeanAndCovariance(label_position, seedling_probability_all_cells[label_position[:,0], label_position[:,1]])
                    
                # Compute distances between reference positions and detected seedlings
                ref_to_seedling_distances_mm = np.zeros((len(seedling_ref_positions_pixels.reshape(-1,2)), number_of_seedling))
                for reference_index in range(NUM_SEEDLING_REFERENCES):
                    ref_to_seedling_distances_mm[reference_index] = calculateEuclidean(seedling_ref_positions_pixels.reshape(-1,2)[reference_index] * mm_per_pixel_roi, dbscan_position_means * mm_per_pixel_roi)
                # Perform linear assignment to match detected seedlings with reference positions
                
                reference_number, seedling_number = linear_sum_assignment(ref_to_seedling_distances_mm)
                
                # Initialize arrays to store seedling statuses, mean positions, covariances, and distances
                seedling_status = np.zeros(NUM_SEEDLING_REFERENCES)
                seedling_position_means = np.zeros((NUM_SEEDLING_REFERENCES, 2))
                seedling_position_covariance = np.zeros((NUM_SEEDLING_REFERENCES, 2, 2))
                seedling_position_to_ref_distance_mm = np.zeros(NUM_SEEDLING_REFERENCES)
                
                # Update seedling status and position data for matched seedlings
                seedling_status[reference_number] = 1
                seedling_position_means[reference_number] = dbscan_position_means[seedling_number]
                seedling_position_covariance[reference_number] = dbscan_position_covariances[seedling_number]
                seedling_position_to_ref_distance_mm[reference_number] = ref_to_seedling_distances_mm[reference_number, seedling_number]    # mm
                seedling_area = calculateEllipseArea(seedling_position_covariance) * mm_per_pixel_roi.prod()                  # mm2
                
                # Identify seedlings based on different conditions
                have_seedling = np.where(seedling_status == 1)[0] 
                missing_seedling = np.where(seedling_status == 0)[0] 
                far_seedling = np.where(seedling_position_to_ref_distance_mm > MAX_SEEDLING_OFFSET_DISTANCE)[0] 
                small_seedling = np.where(np.logical_and(seedling_area < MIN_SEEDLING_SIZE, seedling_area != 0))[0] 
                # Prepare report text based on identified seedlings
                np.set_printoptions(precision=2)                
                report_title = f'{type_of_lettuce}/{image_filename_list[image_index]}\nSeedling  :  {len(have_seedling)}  {np.array2string(have_seedling, separator=', ')}\nSize  :  {np.array2string(seedling_area[have_seedling], separator=', ')}  mm2\nMissing  :  {len(missing_seedling)}  {np.array2string(missing_seedling, separator=', ')}   |   Far  :  {len(far_seedling)} {np.array2string(far_seedling, separator=', ')}   |   Small  :  {len(small_seedling)}  {np.array2string(small_seedling, separator=', ')}'
                # Display the ellipse around each seedling in the tray image
                ellipse_image = displayEllipseOnPlantingTray(rgb_image_roi, dbscan_position_means, dbscan_position_covariances)
                plt.subplot(num_images, 5, (image_index)*5+3), plt.imshow(ellipse_image), plt.title(report_title, loc= 'left')
                
                # sum_have += len(have_seedling)
                # sum_miss += len(missing_seedling)
                # sum_small += len(small_seedling)
                # sum_far += len(far_seedling)
                

                # Assign labels for detected seedlings based on reference numbers
                dbscan_position_labels = np.ones_like(labels) * -1
                for reference_index in range(number_of_seedling):
                    dbscan_position_labels[labels == seedling_number[reference_index]] = reference_number[reference_index]
                
            else:
                # Handle case with no seedlings by setting labels to display empty tray
                dbscan_position_labels = labels

            # Display detected seedling clusters with colors representing their labels
            plt.subplot(num_images, 5, (image_index)*5+2), plt.scatter(seedling_pixel_position[:, 1], seedling_pixel_position[:, 0], c=dbscan_position_labels, cmap='jet', s=0.01), plt.xlim([0, IMG_WIDTH_PIXELS]), plt.ylim([0, IMG_HEIGHT_PIXELS]), plt.colorbar()
            plt.gca().invert_yaxis(), plt.title('clustering'), plt.gca().set_aspect('equal')

        else:
            # Set number_of_seedling to zero if no seedlings are detected
            number_of_seedling = 0
        
        if number_of_seedling == 0:
            report_title = f'{type_of_lettuce}/{image_filename_list[image_index]}\nEmpty tray'
            plt.subplot(num_images, 5, (image_index)*5+3), plt.imshow(rgb_image_roi), plt.title(report_title, loc= 'left')
            # sum_miss += 6
 
        # Display the probability map of seedlings in the tray
        plt.subplot(num_images, 5, (image_index)*5+1), plt.imshow(seedling_probability_all_cells, 'jet'), plt.title(f'segmentation'), plt.colorbar()
    # print(f'{sum_have}, {sum_miss}, {sum_far}, {sum_small}')

### Main2

In [ ]:
# LETTUCE_VARIETY_FOLDERS = ['redbutterhead']
for type_of_lettuce in LETTUCE_VARIETY_FOLDERS:
    
    # sum_have, sum_miss, sum_far, sum_small = 0, 0, 0, 0
    
    # Set the number of Gaussian models based on lettuce type    
    if type_of_lettuce == 'greencos':
        NUM_GAUSSIAN_MODELS = 4
    elif type_of_lettuce == 'redcos':
        NUM_GAUSSIAN_MODELS = 7
    else:
        NUM_GAUSSIAN_MODELS = 5
    
    # Get a sorted list of image names in the folder
    image_filename_list = os.listdir(f'{DATASET_FOLDER_PATH}{type_of_lettuce}/')
    image_filename_list.sort() 
    num_images = len(image_filename_list)

    
    plt.figure(figsize=(42,20))
    for image_index in range(num_images):
        
        # Read the image
        rgb_image = plt.imread( f'{DATASET_FOLDER_PATH}{type_of_lettuce}/{image_filename_list[image_index]}')

        # Decode QR code from the image
        decoded_qr_results = decodeQR(rgb_image)

        # Initialize positions for tray corners for perspective transformation and define QR order
        qr_centroid_positions = np.zeros((4,2), dtype=int)
        qr_order = [3, 1, 2, 4]
        
        # Store distances between QR code corners
        qr_corner_distances_pixels = []
        for qr_index in range(len(decoded_qr_results)):
            
            # Decode QR data and find its centroid position
            data = np.array(decoded_qr_results[qr_index].data.decode("utf-8").split('.'), dtype=int)
            centroid = np.array(decoded_qr_results[qr_index].polygon, dtype=int).mean(axis=0).astype(int)
            qr_centroid_positions[qr_order.index(data[1])] = centroid
            
            # Calculate distances for each corner and store them
            corner = np.array(decoded_qr_results[qr_index].polygon, dtype=int)
            for corner_index in range (corner.shape[0]):
                qr_corner_distances_pixels.append(calculateEuclidean(corner[corner_index], corner[(corner_index+1)%corner.shape[0]]))
       
        # Calculate millimeters per pixel for full image based on QR size
        mm_per_pixel_full_image = QR_SIZE_MM / np.mean(qr_corner_distances_pixels)

        # Calculate pixel distances between tray corners and convert to real distance
        tray_corner_distances_pixels = []
        for corner_index in range (len(decoded_qr_results)):
            tray_corner_distances_pixels.append(calculateEuclidean(qr_centroid_positions[corner_index], qr_centroid_positions[(corner_index+1)%qr_centroid_positions.shape[0]]))
        tray_corner_distances_mm = np.array(tray_corner_distances_pixels) * mm_per_pixel_full_image
        tray_corner_distances_mm = ((tray_corner_distances_mm[:2] + tray_corner_distances_mm[2:]) / 2)
        tray_corner_distances_mm.sort()
        
        # Calculate mm per pixel specifically for the region of interest (ROI)
        mm_per_pixel_roi = (tray_corner_distances_mm) / np.array([IMG_HEIGHT_PIXELS, IMG_WIDTH_PIXELS])

        # Calculate shifts for each QR code based on actual shift distances and convert to pixel units
        qr_shift_pixels = (QR_SHIFT_REAL_DISTANCE / mm_per_pixel_roi).round().astype(int)
        
        # Apply perspective transformation to warp the original image to the region of interest (ROI) defined by the QR code positions.
        rgb_image_roi = warpPerspectiveTransformation(rgb_image, qr_centroid_positions, IMG_HEIGHT_PIXELS, IMG_WIDTH_PIXELS)

        # Convert the ROI image from RGB to YCbCr color space
        ycbcr_image_roi = convertRGBtoYCbCr(rgb_image_roi)

        # Calculate distances from tray edges for reference
        tray_edge_distances_mm = ((TRAY_DIMENSIONS_MM - tray_corner_distances_mm) / 2)

        # Calculate pixel positions for rows and columns of seedling reference positions in the ROI
        seedling_ref_row_positions_pixels = ((SEEDLING_REF_ROW_POS_MM - tray_edge_distances_mm[0]) / mm_per_pixel_roi[0]).round().astype(int)
        seedling_ref_col_positions_pixels = ((SEEDLING_REF_COL_POS_MM - tray_edge_distances_mm[1]) / mm_per_pixel_roi[1]).round().astype(int)
        seedling_ref_positions_pixels = np.stack(np.meshgrid(seedling_ref_row_positions_pixels, seedling_ref_col_positions_pixels), axis=-1).transpose((1,0,2))

        # Set prior probability in spatial domain and zero in corners due to QR shift
        prior_position_tray = np.ones((IMG_HEIGHT_PIXELS, IMG_WIDTH_PIXELS)) * SEEDLING_POSITION_PRIOR
        prior_position_tray[ : qr_shift_pixels[0]  , : qr_shift_pixels[1]  ] = 0
        prior_position_tray[ : qr_shift_pixels[0]  , - qr_shift_pixels[1] :] = 0
        prior_position_tray[ - qr_shift_pixels[0] :, : qr_shift_pixels[1]  ] = 0
        prior_position_tray[ - qr_shift_pixels[0] :, - qr_shift_pixels[1] :] = 0        
        prior_position_tray[: qr_shift_pixels[0] // 3  ] = 0
        prior_position_tray[- qr_shift_pixels[0] // 3 :] = 0
        prior_position_tray[: , : qr_shift_pixels[1] // 3  ] = 0
        prior_position_tray[: , - qr_shift_pixels[1] // 3 :] = 0

        # Load color models from seedling tray for seedlings and background
        seedling_color_mean = np.loadtxt(f'D:/#Work/4-1 WiL/Workspace/Dataset/planting_tray/seedling_tray/phone/{type_of_lettuce}_mean.npy')
        seedling_color_covariance = np.loadtxt(f'D:/#Work/4-1 WiL/Workspace/Dataset/planting_tray/seedling_tray/phone/{type_of_lettuce}_covariance.npy')
        background_color_means = np.load(f'D:/#Work/4-1 WiL/Workspace/Dataset/planting_tray/seedling_tray/phone/background_{NUM_GAUSSIAN_MODELS-1}_means.npy')
        background_color_covariances = np.load(f'D:/#Work/4-1 WiL/Workspace/Dataset/planting_tray/seedling_tray/phone/background_{NUM_GAUSSIAN_MODELS-1}_covariances.npy')

        # Combine seedling and background color models for the tray
        seedling_tray_color_model_means = np.vstack([seedling_color_mean, background_color_means])
        seedling_tray_color_model_covariances = np.vstack([np.expand_dims(seedling_color_covariance, axis=0), background_color_covariances])

        # Fit a Gaussian Mixture Model (GMM) to the color space of the ROI image
        gmm = GaussianMixture(n_components= NUM_GAUSSIAN_MODELS )
        gmm.fit(ycbcr_image_roi.reshape(-1,3))
        gmm_means = gmm.means_ 
        gmm_covariances = gmm.covariances_
        
        # Align GMM models with existing color models, ensuring the seedling model is first
        pair_gmm_indices = pairGaussianModels(seedling_tray_color_model_means, seedling_tray_color_model_covariances, gmm_means, gmm_covariances)
        gmm_means = gmm_means[pair_gmm_indices[1]]
        gmm_covariances = gmm_covariances[pair_gmm_indices[1]]

        # Compute likelihoods for seedling and background using the GMMs and color models
        pdf_seedling_color_model = calculateMultiGaussianDist(ycbcr_image_roi, seedling_tray_color_model_means[0], seedling_tray_color_model_covariances[0])
        pdf_seedling_gmm = calculateMultiGaussianDist(ycbcr_image_roi, gmm_means[0], gmm_covariances[0])
        foreground_likelihood = pdf_seedling_color_model * pdf_seedling_gmm

        background_likelihood = np.zeros((IMG_HEIGHT_PIXELS, IMG_WIDTH_PIXELS))
        for index_background_clusters in range (1, NUM_GAUSSIAN_MODELS):
            pdf_background_color_model = calculateMultiGaussianDist(ycbcr_image_roi, seedling_tray_color_model_means[index_background_clusters], seedling_tray_color_model_covariances[index_background_clusters])
            pdf_background_gmm = calculateMultiGaussianDist(ycbcr_image_roi, gmm_means[index_background_clusters], gmm_covariances[index_background_clusters])
            background_likelihood += pdf_background_color_model * pdf_background_gmm 

        # Calculate the seedling probability for each cell using Bayes' theorem
        seedling_probability_all_cells = calculateBayes(foreground_likelihood * prior_position_tray, background_likelihood * (1 - prior_position_tray) / (NUM_GAUSSIAN_MODELS-1))

        # Identify pixel positions with a high likelihood of containing seedlings
        seedling_pixel_position = np.stack(np.where(seedling_probability_all_cells > DBSCAN_PROBABILITY_THRESHOLD), axis = -1)

        # Check if there are detected seedlings; if so, apply DBSCAN for clustering
        if len(seedling_pixel_position) != 0:
            # Apply DBSCAN clustering on seedling pixel positions to group seedlings
            dbscan = DBSCAN(eps = DBSCAN_MAX_DIST_MM, min_samples = DBSCAN_MIN_POINTS)
            labels = dbscan.fit_predict(seedling_pixel_position)
            # Calculate the number of distinct seedlings
            number_of_seedling = labels.max() + 1
            if number_of_seedling != 0:
                # Initialize arrays to store means and covariances for each detected seedling
                dbscan_position_means = np.zeros((number_of_seedling, 2))
                dbscan_position_covariances = np.zeros((number_of_seedling, 2, 2))

                # Loop through each detected seedling cluster
                for label_index in range(number_of_seedling):
                    # Get positions of the pixels for the current seedling cluster
                    label_position = seedling_pixel_position[np.where(labels == label_index)]
                    # Calculate mean and covariance for current seedling cluster based on probabilities
                    dbscan_position_means[label_index], dbscan_position_covariances[label_index] = calculateMeanAndCovariance(label_position, seedling_probability_all_cells[label_position[:,0], label_position[:,1]])
                    
                # Compute distances between reference positions and detected seedlings
                ref_to_seedling_distances_mm = np.zeros((len(seedling_ref_positions_pixels.reshape(-1,2)), number_of_seedling))
                for reference_index in range(NUM_SEEDLING_REFERENCES):
                    ref_to_seedling_distances_mm[reference_index] = calculateEuclidean(seedling_ref_positions_pixels.reshape(-1,2)[reference_index] * mm_per_pixel_roi, dbscan_position_means * mm_per_pixel_roi)
                # Perform linear assignment to match detected seedlings with reference positions
                
                reference_number, seedling_number = linear_sum_assignment(ref_to_seedling_distances_mm)
                
                # Initialize arrays to store seedling statuses, mean positions, covariances, and distances
                seedling_status = np.zeros(NUM_SEEDLING_REFERENCES)
                seedling_position_means = np.zeros((NUM_SEEDLING_REFERENCES, 2))
                seedling_position_covariance = np.zeros((NUM_SEEDLING_REFERENCES, 2, 2))
                seedling_position_to_ref_distance_mm = np.zeros(NUM_SEEDLING_REFERENCES)
                
                # Update seedling status and position data for matched seedlings
                seedling_status[reference_number] = 1
                seedling_position_means[reference_number] = dbscan_position_means[seedling_number]
                seedling_position_covariance[reference_number] = dbscan_position_covariances[seedling_number]
                seedling_position_to_ref_distance_mm[reference_number] = ref_to_seedling_distances_mm[reference_number, seedling_number]    # mm
                seedling_area = calculateEllipseArea(seedling_position_covariance) * mm_per_pixel_roi.prod()                  # mm2
                
                # Identify seedlings based on different conditions
                have_seedling = np.where(seedling_status == 1)[0] 
                missing_seedling = np.where(seedling_status == 0)[0] 
                far_seedling = np.where(seedling_position_to_ref_distance_mm > MAX_SEEDLING_OFFSET_DISTANCE)[0] 
                small_seedling = np.where(np.logical_and(seedling_area < MIN_SEEDLING_SIZE, seedling_area != 0))[0] 
                # Prepare report text based on identified seedlings
                np.set_printoptions(precision=2)                
                report_title = f'{type_of_lettuce}/{image_filename_list[image_index]}\nSeedling  :  {len(have_seedling)}  {np.array2string(have_seedling, separator=', ')}\nSize  :  {np.array2string(seedling_area[have_seedling], separator=', ')}  mm2\nMissing  :  {len(missing_seedling)}  {np.array2string(missing_seedling, separator=', ')}   |   Far  :  {len(far_seedling)} {np.array2string(far_seedling, separator=', ')}   |   Small  :  {len(small_seedling)}  {np.array2string(small_seedling, separator=', ')}'
                # Display the ellipse around each seedling in the tray image
                ellipse_image = displayEllipseOnPlantingTray(rgb_image_roi, dbscan_position_means, dbscan_position_covariances)
                plt.subplot(3, 5, (image_index)+1), plt.imshow(ellipse_image), plt.title(report_title, loc= 'left')
                
                # sum_have += len(have_seedling)
                # sum_miss += len(missing_seedling)
                # sum_small += len(small_seedling)
                # sum_far += len(far_seedling)
                

                # Assign labels for detected seedlings based on reference numbers
                dbscan_position_labels = np.ones_like(labels) * -1
                for reference_index in range(number_of_seedling):
                    dbscan_position_labels[labels == seedling_number[reference_index]] = reference_number[reference_index]
                
            else:
                # Handle case with no seedlings by setting labels to display empty tray
                dbscan_position_labels = labels

            # Display detected seedling clusters with colors representing their labels
            # plt.subplot(3, 5, (image_index)*5+2), plt.scatter(seedling_pixel_position[:, 1], seedling_pixel_position[:, 0], c=dbscan_position_labels, cmap='jet', s=0.01), plt.xlim([0, IMG_WIDTH_PIXELS]), plt.ylim([0, IMG_HEIGHT_PIXELS]), plt.colorbar()
            # plt.gca().invert_yaxis(), plt.title('clustering'), plt.gca().set_aspect('equal')

        else:
            # Set number_of_seedling to zero if no seedlings are detected
            number_of_seedling = 0
        
        if number_of_seedling == 0:
            report_title = f'{type_of_lettuce}/{image_filename_list[image_index]}\nEmpty tray'
            plt.subplot(3, 5, (image_index)+1), plt.imshow(rgb_image_roi), plt.title(report_title, loc= 'left')
            # sum_miss += 6
 
        # Display the probability map of seedlings in the tray
        # plt.subplot(num_images, 5, (image_index)*5+1), plt.imshow(seedling_probability_all_cells, 'jet'), plt.title(f'segmentation'), plt.colorbar()
    # print(f'{sum_have}, {sum_miss}, {sum_far}, {sum_small}')

### Result

In [ ]:
gc = np.array([69, 21, 4, 0]) * np.identity(4)
rc = np.array([73, 17, 7, 0]) * np.identity(4)
bh = np.array([78, 12, 7, 20]) * np.identity(4)
rbh = np.array([69, 21, 5, 7]) * np.identity(4)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
cm_display = ConfusionMatrixDisplay(confusion_matrix = gc, display_labels = ['Seedling', 'Missing', 'Far', 'Small'])
cm_display.plot(), plt.title('Green Cos')

In [ ]:
cm_display = ConfusionMatrixDisplay(confusion_matrix = rc, display_labels = ['Seedling', 'Missing', 'Far', 'Small'])
cm_display.plot(), plt.title('Red Cos')

In [ ]:
cm_display = ConfusionMatrixDisplay(confusion_matrix = bh, display_labels = ['Seedling', 'Missing', 'Far', 'Small'])
cm_display.plot(), plt.title('Butterhead')

In [ ]:
cm_display = ConfusionMatrixDisplay(confusion_matrix = rbh, display_labels = ['Seedling', 'Missing', 'Far', 'Small'])
cm_display.plot(), plt.title('Red Butterhead')